In [1]:
!pip install rembg
!pip install cog
!pip install diffusers
!pip install transformers
!pip install torch
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.8/90.8 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.4/341.4 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [9]:
import os
import subprocess
from typing import List
import cv2
import rembg
import torch
from cog import BasePredictor, Input, Path
from diffusers import DiffusionPipeline, EulerAncestralDiscreteScheduler
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

WEIGHTS_CACHE = "/src/weights/zero123plusplus"
CHECKPOINT_URLS = [
    ("https://weights.replicate.delivery/default/zero123plusplus/zero123plusplus.tar", WEIGHTS_CACHE),
]



class Predictor(BasePredictor):
    def setup(self) -> None:
        """Load the model into memory to make running multiple predictions efficient"""

        if not os.path.exists("weights"):
            os.mkdir("weights")


        print("Setting up pipeline...")

        self.pipeline = DiffusionPipeline.from_pretrained(
            "/content/drive/MyDrive/TeamProjects/DimensionalPioneers/4.zero123/weights/zero123plusplus",
            custom_pipeline="/content/drive/MyDrive/TeamProjects/DimensionalPioneers/4.zero123/diffusers-support/",
            torch_dtype=torch.float16,
            local_files_only=True
        )
        self.pipeline.scheduler = EulerAncestralDiscreteScheduler.from_config(
            self.pipeline.scheduler.config, timestep_spacing='trailing'
        )
        self.pipeline.to('cuda:0')

    def predict(
        self,
        image: Path = Input(description="Input image. Aspect ratio should be 1:1. Recommended resolution is >= 320x320 pixels."),
        remove_background: bool = Input(description="Remove the background of the input image", default=False),
        return_intermediate_images: bool = Input(description="Return the intermediate images together with the output images", default=False),
    ) -> List[Path]:
        """Run a single prediction on the model"""
        outputs = []

        cond = Image.open(str(image))
        ret, mask = cv2.threshold(np.array(cond.split()[-1]), 0, 255, cv2.THRESH_BINARY)
        x, y, w, h = cv2.boundingRect(mask)
        image_filename = "original" + image.suffix

        # optional background removal step
        if remove_background:
            rembg_session = rembg.new_session()
            cond = rembg.remove(cond, session=rembg_session)
            # image should be a png after background removal
            image_filename += ".png"

        if return_intermediate_images:
            temp_original = f"/tmp/{image_filename}"
            cond.save(temp_original)
            outputs.append(temp_original)

        all_results = self.pipeline(cond, num_inference_steps=50).images[0]
        side_len = all_results.width//2
        subimages = [all_results.crop((x, y, x + side_len, y+side_len)) for y in range(0, all_results.height, side_len) for x in range(0, all_results.width, side_len)]
        for i, output_img in enumerate(subimages):
            filename = f"/content/drive/MyDrive/TeamProjects/DimensionalPioneers/4.zero123/Result/output{i+1}.png"
            output_img.save(filename)
            outputs.append(filename)

        return([Path(output) for output in outputs])


# Predictor 인스턴스 생성
predictor = Predictor()

# 모델 설정
predictor.setup()

# 이미지 변환 실행
result_paths = predictor.predict(image=Path("/content/drive/MyDrive/TeamProjects/DimensionalPioneers/4.zero123/Data/Img/extinguisher.png"), remove_background=True, return_intermediate_images=False)

# 결과 경로 출력
for path in result_paths:
    print(path)


Setting up pipeline...


Loading pipeline components...:   0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

/content/drive/MyDrive/TeamProjects/DimensionalPioneers/4.zero123/Result/output1.png
/content/drive/MyDrive/TeamProjects/DimensionalPioneers/4.zero123/Result/output2.png
/content/drive/MyDrive/TeamProjects/DimensionalPioneers/4.zero123/Result/output3.png
/content/drive/MyDrive/TeamProjects/DimensionalPioneers/4.zero123/Result/output4.png
/content/drive/MyDrive/TeamProjects/DimensionalPioneers/4.zero123/Result/output5.png
/content/drive/MyDrive/TeamProjects/DimensionalPioneers/4.zero123/Result/output6.png


In [ ]:
import os
import subprocess
from typing import List
import cv2
import rembg
import torch
from cog import BasePredictor, Input, Path
from diffusers import DiffusionPipeline, EulerAncestralDiscreteScheduler
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

WEIGHTS_CACHE = "/src/weights/zero123plusplus"
CHECKPOINT_URLS = [
    ("https://weights.replicate.delivery/default/zero123plusplus/zero123plusplus.tar", WEIGHTS_CACHE),
]



class Predictor(BasePredictor):
    def setup(self) -> None:
        """Load the model into memory to make running multiple predictions efficient"""

        if not os.path.exists("weights"):
            os.mkdir("weights")


        print("Setting up pipeline...")

        self.pipeline = DiffusionPipeline.from_pretrained(
            "/content/drive/MyDrive/TeamProjects/DimensionalPioneers/4.zero123/weights/zero123plusplus",
            custom_pipeline="/content/drive/MyDrive/TeamProjects/DimensionalPioneers/4.zero123/diffusers-support/",
            torch_dtype=torch.float16,
            local_files_only=True
        )
        self.pipeline.scheduler = EulerAncestralDiscreteScheduler.from_config(
            self.pipeline.scheduler.config, timestep_spacing='trailing'
        )
        self.pipeline.to('cuda:0')

    def predict(
        self,
        image: Path = Input(description="Input image. Aspect ratio should be 1:1. Recommended resolution is >= 320x320 pixels."),
        remove_background: bool = Input(description="Remove the background of the input image", default=False),
        return_intermediate_images: bool = Input(description="Return the intermediate images together with the output images", default=False),
        output_prefix: int = 1,
      ) -> List[Path]:
        """Run a single prediction on the model"""
        outputs = []

        cond = Image.open(str(image))
        ret, mask = cv2.threshold(np.array(cond.split()[-1]), 0, 255, cv2.THRESH_BINARY)
        x, y, w, h = cv2.boundingRect(mask)
        image_filename = "original" + image.suffix

        # optional background removal step
        if remove_background:
            rembg_session = rembg.new_session()
            cond = rembg.remove(cond, session=rembg_session)
            # image should be a png after background removal
            image_filename += ".png"

        if return_intermediate_images:
            temp_original = f"/tmp/{image_filename}"
            cond.save(temp_original)
            outputs.append(temp_original)

        all_results = self.pipeline(cond, num_inference_steps=50).images[0]
        side_len = all_results.width//2
        subimages = [all_results.crop((x, y, x + side_len, y+side_len)) for y in range(0, all_results.height, side_len) for x in range(0, all_results.width, side_len)]
        for i, output_img in enumerate(subimages):
            filename = f"/content/drive/MyDrive/TeamProjects/DimensionalPioneers/4.zero123/reinforced_images/output{output_prefix}_{i+1}.png"
            output_img.save(filename)
            outputs.append(filename)

        return([Path(output) for output in outputs])


# Predictor 인스턴스 생성
predictor = Predictor()

# 모델 설정
predictor.setup()

for j in range(10):
  # 이미지 변환 실행
  result_paths = predictor.predict(image=Path(f"/content/drive/MyDrive/TeamProjects/DimensionalPioneers/4.zero123/Data/Img/img006.jpg"), remove_background=True, return_intermediate_images=False, output_prefix=j+1)
  # 결과 경로 출력
  for path in result_paths:
      print(path)

Setting up pipeline...


Loading pipeline components...:   0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

/content/drive/MyDrive/TeamProjects/DimensionalPioneers/4.zero123/reinforced_images/output1_1.png
/content/drive/MyDrive/TeamProjects/DimensionalPioneers/4.zero123/reinforced_images/output1_2.png
/content/drive/MyDrive/TeamProjects/DimensionalPioneers/4.zero123/reinforced_images/output1_3.png
/content/drive/MyDrive/TeamProjects/DimensionalPioneers/4.zero123/reinforced_images/output1_4.png
/content/drive/MyDrive/TeamProjects/DimensionalPioneers/4.zero123/reinforced_images/output1_5.png
/content/drive/MyDrive/TeamProjects/DimensionalPioneers/4.zero123/reinforced_images/output1_6.png


  0%|          | 0/50 [00:00<?, ?it/s]

...

/content/drive/MyDrive/TeamProjects/DimensionalPioneers/4.zero123/reinforced_images/output8_1.png
/content/drive/MyDrive/TeamProjects/DimensionalPioneers/4.zero123/reinforced_images/output8_2.png
/content/drive/MyDrive/TeamProjects/DimensionalPioneers/4.zero123/reinforced_images/output8_3.png
/content/drive/MyDrive/TeamProjects/DimensionalPioneers/4.zero123/reinforced_images/output8_4.png
/content/drive/MyDrive/TeamProjects/DimensionalPioneers/4.zero123/reinforced_images/output8_5.png
/content/drive/MyDrive/TeamProjects/DimensionalPioneers/4.zero123/reinforced_images/output8_6.png


  0%|          | 0/50 [00:00<?, ?it/s]

/content/drive/MyDrive/TeamProjects/DimensionalPioneers/4.zero123/reinforced_images/output9_1.png
/content/drive/MyDrive/TeamProjects/DimensionalPioneers/4.zero123/reinforced_images/output9_2.png
/content/drive/MyDrive/TeamProjects/DimensionalPioneers/4.zero123/reinforced_images/output9_3.png
/content/drive/MyDrive/TeamProjects/DimensionalPioneers/4.zero123/reinforced_images/output9_4.png
/content/drive/MyDrive/TeamProjects/DimensionalPioneers/4.zero123/reinforced_images/output9_5.png
/content/drive/MyDrive/TeamProjects/DimensionalPioneers/4.zero123/reinforced_images/output9_6.png


  0%|          | 0/50 [00:00<?, ?it/s]

/content/drive/MyDrive/TeamProjects/DimensionalPioneers/4.zero123/reinforced_images/output10_1.png
/content/drive/MyDrive/TeamProjects/DimensionalPioneers/4.zero123/reinforced_images/output10_2.png
/content/drive/MyDrive/TeamProjects/DimensionalPioneers/4.zero123/reinforced_images/output10_3.png
/content/drive/MyDrive/TeamProjects/DimensionalPioneers/4.zero123/reinforced_images/output10_4.png
/content/drive/MyDrive/TeamProjects/DimensionalPioneers/4.zero123/reinforced_images/output10_5.png
/content/drive/MyDrive/TeamProjects/DimensionalPioneers/4.zero123/reinforced_images/output10_6.png


In [ ]:
import torch
import requests
from PIL import Image
from diffusers import DiffusionPipeline, EulerAncestralDiscreteScheduler, ControlNetModel

# Load the pipeline
pipeline = DiffusionPipeline.from_pretrained(
    "sudo-ai/zero123plus-v1.1", custom_pipeline="sudo-ai/zero123plus-pipeline",
    torch_dtype=torch.float16
)
pipeline.add_controlnet(ControlNetModel.from_pretrained(
    "sudo-ai/controlnet-zp11-depth-v1",
    torch_dtype=torch.float16
), conditioning_scale=0.75)
# Feel free to tune the scheduler
pipeline.scheduler = EulerAncestralDiscreteScheduler.from_config(
    pipeline.scheduler.config, timestep_spacing='trailing'
)
pipeline.to('cuda:0')
# Run the pipeline
cond = Image.open("/content/drive/MyDrive/TeamProjects/DimensionalPioneers/4.zero123/Data/Img/img006.jpg")
depth = Image.open("/content/drive/MyDrive/TeamProjects/DimensionalPioneers/4.zero123/Result/output0.jpg")
result = pipeline(cond, depth_image=depth, num_inference_steps=36).images[0]
result.show()
result.save("output.png")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/2.57k [00:00<?, ?B/s]

pipeline.py:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/8 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

  0%|          | 0/36 [00:00<?, ?it/s]

In [ ]:
import numpy
from PIL import Image
from pymatting.alpha.estimate_alpha_cf import estimate_alpha_cf
from pymatting.foreground.estimate_foreground_ml import estimate_foreground_ml
from pymatting.util.util import stack_images
from scipy.ndimage import binary_erosion
from typing import Tuple


def postprocess(rgb_img: Image.Image, normal_img: Image.Image) -> Tuple[Image.Image, Image.Image]:
    normal_vecs_pred = numpy.array(normal_img, dtype=numpy.float64) / 255.0 * 2 - 1
    alpha_pred = numpy.linalg.norm(normal_vecs_pred, axis=-1)

    is_foreground = alpha_pred > 0.6
    is_background = alpha_pred < 0.2
    structure = numpy.ones(
        (4, 4), dtype=numpy.uint8
    )

    is_foreground = binary_erosion(is_foreground, structure=structure)
    is_background = binary_erosion(is_background, structure=structure, border_value=1)

    trimap = numpy.full(alpha_pred.shape, dtype=numpy.uint8, fill_value=128)
    trimap[is_foreground] = 255
    trimap[is_background] = 0

    img_normalized = numpy.array(rgb_img, dtype=numpy.float64) / 255.0
    trimap_normalized = trimap.astype(numpy.float64) / 255.0

    alpha = estimate_alpha_cf(img_normalized, trimap_normalized)
    foreground = estimate_foreground_ml(img_normalized, alpha)
    cutout = stack_images(foreground, alpha)

    cutout = numpy.clip(cutout * 255, 0, 255).astype(numpy.uint8)
    cutout = Image.fromarray(cutout)

    normal_vecs_pred = normal_vecs_pred / (numpy.linalg.norm(normal_vecs_pred, axis=-1, keepdims=True) + 1e-8)
    normal_vecs_pred = normal_vecs_pred * 0.5 + 0.5
    normal_vecs_pred = normal_vecs_pred * alpha[..., None] + 0.5 * (1 - alpha[..., None])
    normal_image_normalized = numpy.clip(normal_vecs_pred * 255, 0, 255).astype(numpy.uint8)

    return cutout, Image.fromarray(normal_image_normalized)

In [ ]:
import cv2
import copy
import numpy
import torch
import requests
from PIL import Image
from diffusers import DiffusionPipeline, ControlNetModel
# from matting_postprocess import postprocess


def rescale(single_res, input_image, ratio=0.95):
    # Rescale and recenter
    image_arr = numpy.array(input_image)
    ret, mask = cv2.threshold(numpy.array(input_image.split()[-1]), 0, 255, cv2.THRESH_BINARY)
    x, y, w, h = cv2.boundingRect(mask)
    max_size = max(w, h)
    side_len = int(max_size / ratio)
    padded_image = numpy.zeros((side_len, side_len, 4), dtype=numpy.uint8)
    center = side_len//2
    padded_image[center-h//2:center-h//2+h, center-w//2:center-w//2+w] = image_arr[y:y+h, x:x+w]
    rgba = Image.fromarray(padded_image).resize((single_res, single_res), Image.LANCZOS)
    return rgba


# Load the pipeline
pipeline = DiffusionPipeline.from_pretrained(
    "sudo-ai/zero123plus-v1.1", custom_pipeline="sudo-ai/zero123plus-pipeline",
    torch_dtype=torch.float16
)
normal_pipeline = copy.copy(pipeline)
normal_pipeline.add_controlnet(ControlNetModel.from_pretrained(
    "sudo-ai/controlnet-zp12-normal-gen-v1", torch_dtype=torch.float16, local_files_only=False
), conditioning_scale=1.0)
pipeline.to("cuda:0", torch.float16)
normal_pipeline.to("cuda:0", torch.float16)
# Run the pipeline
cond = Image.open(requests.get("https://d.skis.ltd/nrp/sample-data/10_cond.png", stream=True).raw)
# Optional: rescale input image if it occupies only a small region in input
# cond = rescale(512, cond)
# Generate 6 images
genimg = pipeline(
    cond,
    prompt='', guidance_scale=4, num_inference_steps=75, width=640, height=960
).images[0]
# Generate normal image
# We observe that a higher CFG scale (4) is more robust
# but with CFG = 1 it is faster and is usually good enough for normal image
# You can adjust to your needs
normalimg = normal_pipeline(
    cond, depth_image=genimg,
    prompt='', guidance_scale=4, num_inference_steps=75, width=640, height=960
).images[0]
genimg, normalimg = postprocess(genimg, normalimg)
ret, mask = cv2.threshold(numpy.array(cond.split()[-1]), 0, 255, cv2.THRESH_BINARY)
x, y, w, h = cv2.boundingRect(mask)
side_len = genimg.width//2
subimages = [normalimg.crop((x, y, x + side_len, y+side_len)) for y in range(0, normalimg.height, side_len) for x in range(0, normalimg.width, side_len)]
for i, output_img in enumerate(subimages):
    filename = f"/content/drive/MyDrive/TeamProjects/DimensionalPioneers/4.zero123/Result/output{i+1}.png"
    output_img.save(filename)
genimg.save("colors.png")
normalimg.save("normals.png")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/2.57k [00:00<?, ?B/s]

pipeline.py:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

...

Loading pipeline components...:   0%|          | 0/8 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

  0%|          | 0/75 [00:00<?, ?it/s]

KeyboardInterrupt: 